In [1]:
import transformers
import jsonlines
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import tqdm
import torch.optim as optim
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BlenderbotForConditionalGeneration,
    BlenderbotTokenizer,
)

In [2]:
train_path = './dataset.jsonl'

In [3]:
train_data = []
with jsonlines.open(train_path) as f:
    for line in f.iter():
        train_data.append(line)
# val_data = []
# with jsonlines.open(val_path) as f:
#     for line in f.iter():
#         val_data.append(line)

In [4]:
#tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
#model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")
device = "cuda" if torch.cuda.is_available() else "cpu"
mname = "facebook/blenderbot-400M-distill"
model = BlenderbotForConditionalGeneration.from_pretrained(mname).to(device)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)



# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
# model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small").to(device)

In [5]:
# def preprocess_function(examples):
#     encode_examples = {}
#     inputs = []
#     targets = []
#     for doc in examples:
#         for i, dialog in enumerate(doc['dialog']):
#             if i%2 == 1:
#                 inputs.append(dialog)
#             else:
#                 targets.append(dialog)
#     # inputs = [doc["maintext"] for doc in examples]
#     tokenizer.pad_token = tokenizer.eos_token
#     encode_examples = tokenizer(inputs, max_length=100, truncation=True, padding=True)
#     encode_examples['target'] = tokenizer(targets, max_length=100, truncation=True, padding=True)
    
#     return encode_examples

In [23]:
def preprocess_function(examples):
    encode_examples = {}
    inputs = []
    targets = []
    for doc in examples:
        for dialog in doc['source']:
            inputs.append(dialog)
        for dialog in doc['target']: 
            targets.append(dialog)
            
    # inputs = [doc["maintext"] for doc in examples]
    encode_examples['source']  = tokenizer(inputs, max_length=200, truncation=True, padding=True)
    encode_examples['target'] = tokenizer(targets, max_length=64, truncation=True, padding=True)
    return encode_examples

In [24]:
tokenized_summerize = {}
tokenized_set = preprocess_function(train_data)
#tokenized_summerize['validation'] = preprocess_function(val_data)

In [26]:
print(len(tokenized_set['target']['input_ids']))
print(len(tokenized_set['source']['input_ids']))

639112
1594153


In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
if torch.cuda.is_available() == True:
    device = 'cuda'
else:
    device = 'cpu'

In [9]:
class Dialog(Dataset):
    def __init__(self, encoded_dataset):
        self.token = encoded_dataset['input_ids']
        self.label = encoded_dataset['target']['input_ids'] if 'target' in encoded_dataset.keys() else None
    def __getitem__(self, index):
        if self.label is None:
            return torch.tensor(self.token[index])
        else:
            return torch.tensor(self.token[index]), torch.tensor(self.label[index])
    def __len__(self):
        return len(self.token)    

In [10]:
trainset = Dialog(tokenized_set)
#valset = Dialog(tokenized_summerize['validation'])
trainloader = DataLoader(dataset = trainset, batch_size = 64, shuffle = True)
#valloader = DataLoader(dataset = valset, batch_size = 16, shuffle = False)

In [11]:
def save_checkpoint(checkpoint_path, model, optimizer):
    #torch.save(model.state_dict(), checkpoint_path)
    model.save_pretrained(checkpoint_path)
    tokenizer.save_pretrained(checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    model = torch.load(checkpoint_path)
    #model.load_state_dict(state['state_dict'])
    print('model loaded from %s' % checkpoint_path)

In [12]:
# rouge_list = []
# def test(model):
#     model.eval()
#     label_list,eval_predictions = [], []
#     with torch.no_grad():
#         for i,(inputs_ids, labels) in enumerate(valloader):
#             if i%170 == 0 and i != 0:
#                 print(f"---start evaluation---")
#             inputs_ids, labels = inputs_ids.to(device), labels.to(device)
#             outputs = model.generate(inputs_ids,num_beams=5,max_length=64)
#             eval_predictions  += outputs
#             label_list += labels
#         rouge = compute_metrics(eval_predictions, label_list)
#         print(rouge)
#         rouge_list.append(rouge)

In [13]:
def train_save(model, epoch, save_interval, log_interval=100):
    optimizer = optim.AdamW(model.parameters(),lr=1e-4, betas=(0.9, 0.999),weight_decay=0)
    model.train()  # set training mode
    best = 0    
    iteration = 0
    for ep in range(epoch):
        loss_sum = 0
        for batch_idx, (input_ids, labels) in enumerate(trainloader):
            input_ids, labels = input_ids.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(input_ids=input_ids,labels=labels)
            loss = output.loss
            loss_sum = loss_sum + loss.item()
            loss.backward()
            optimizer.step()
            
            if iteration % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(input_ids), len(trainloader.dataset),
                    100. * batch_idx / len(trainloader), loss.item()))
            if iteration % save_interval == 0 and iteration > 0:
                # test(model)
                if iteration>500:
                    save_checkpoint('./summary-%i/' % iteration, model, optimizer)    
            
            iteration += 1
            
        print('\nTrain set:Loss: ({:.0f})\n'.format(loss_sum / len(trainloader)))
        
    
    # save the final model
    save_checkpoint('./summary-%i/' % iteration, model, optimizer)

In [14]:
model = model.to(device)
train_save(model, epoch = 8, save_interval = 500, log_interval = 100)

IndexError: list index out of range